# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-6be66ecdd5-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series ("Olympic Games explorative search") 

Consider the following exploratory information need:

> investigate the Olympic Games and find the main BGPs related to these events. Find all the editions of the Summer and Winter Games, compare the sports and disciplines that belong to the games, records and countries.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`       | Human         | node      |
| `wd:Q1189`    | Usain Bolt    | node      |





Also consider

```
wd:Q1189 ?p ?obj .
```

is the BGP to retrieve all **properties of Usain Bolt**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.


The workload should:


1. Identify the BGP for Olympic Games

2. Return all the editions of the Summer Olympic Games (do not consider future Olympic Games) with the country where they were played (the result set must be a list of elements with edition IRI and label, and country IRI and label).

3. Find the countries which held more than 2 Olympic Games (both Summer and Winter Games, do not consider future Olympic Games) (the result set must be a list of triples country IRI, label and #edition held).

4. Consider the 2008 Summer Olympic Games. Identify all the sports played in this edition (the result set must be a list of couples sport IRI and label).

5. Consider the 2008 Summer Olympic Games. For each sport return the number of different disciplines (the result set must be a list of triples sport IRI, label and #disciplines).

6. Consider the 2008 Summer Olympic Games. Find the top-10 people who won more gold medals (the result set must be a list of triples athlete IRI, label and #gold medal).

7. Find athletes who won at least one gold medal in 4 different editions of the Summer Olympic Games (the result set must be a list of triples athlete IRI, label and #edition).

8. Consider all the edition of the Summer Olympic Games. For each sport return the athlete who won more gold medal (the result set must be a list of elements with sport IRI and label, athlete IRI and label, and #gold medal). 

## Task 1
Identify the BGP for Olympic Games

In [10]:
# query example
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?oname
WHERE {
   # bind something
   wd:Q1189 ?p ?obj .
   # get the label
   ?p sc:name ?pname.
   ?obj sc:name ?oname
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P54'), ('pname', 'member of sports team'), ('obj', 'http://www.wikidata.org/entity/Q762294'), ('oname', 'Central Coast Mariners FC')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pname', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q55256167'), ('oname', "2013 Zürich Weltklasse – men's 100 metres")]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pname', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q55256874'), ('oname', "2012 Lausanne Athletissima – men's 200 metres")]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pname', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q55256933'), ('oname', "2012 Zürich Weltklasse – men's 200 metres")]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pname', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q55257847'), ('oname', "2010 Lausanne Athletissima – men's 100 metres")]
[('p', 'http://www.wikidata.org/prop/direc

In [11]:
# Q1053559 --> '2020 Winter Youth Olympics'
queryString = """
SELECT  ?s ?sname ?pname
WHERE {
   # bind something
   ?s ?p wd:Q1053559 .
   # get the label
   ?s sc:name ?sname .
   ?p sc:name ?pname.
   #?o sc:name ?oname
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q85816744'), ('sname', 'Yodli'), ('pname', 'represents')]
[('s', 'http://www.wikidata.org/entity/Q81904264'), ('sname', 'Kim Ji-min'), ('pname', 'participant in')]
[('s', 'http://www.wikidata.org/entity/Q76449094'), ('sname', 'Victoria Stirnemann'), ('pname', 'participant in')]
[('s', 'http://www.wikidata.org/entity/Q81904099'), ('sname', 'Benjamin Kapp'), ('pname', 'participant in')]
[('s', 'http://www.wikidata.org/entity/Q81902447'), ('sname', 'Alexander Czudaj'), ('pname', 'participant in')]
[('s', 'http://www.wikidata.org/entity/Q15080578'), ('sname', 'Marcus Cyron'), ('pname', 'participant in')]
[('s', 'http://www.wikidata.org/entity/Q65658519'), ('sname', 'Sandro Halank'), ('pname', 'participant in')]
[('s', 'http://www.wikidata.org/entity/Q81906761'), ('sname', 'William Scammell'), ('pname', 'participant in')]
[('s', 'http://www.wikidata.org/entity/Q66707403'), ('sname', 'Jeffrey Chen'), ('pname', 'participant in')]
[('s', 'http://w

In [14]:
# P1344 --> 'participant in'
queryString = """
SELECT ?sname ?o ?oname
WHERE {
   # bind something
   ?s wdt:P1344 ?o .
   # get the label
   ?s sc:name ?sname .
   #?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('sname', 'Kim Ji-min'), ('o', 'http://www.wikidata.org/entity/Q1053559'), ('oname', '2020 Winter Youth Olympics')]
[('sname', 'Robert Foth'), ('o', 'http://www.wikidata.org/entity/Q17088224'), ('oname', "shooting at the 1992 Summer Olympics – men's 50 metre rifle three positions")]
[('sname', 'Han Tianyu'), ('o', 'http://www.wikidata.org/entity/Q15506507'), ('oname', "short track speed skating at the 2014 Winter Olympics – men's 1500 metres")]
[('sname', 'Shi Yuqi'), ('o', 'http://www.wikidata.org/entity/Q15741854'), ('oname', "2014 Asian Junior Badminton Championships – men's singles")]
[('sname', 'Irina Gerasimenok'), ('o', 'http://www.wikidata.org/entity/Q7500802'), ('oname', "shooting at the 1996 Summer Olympics – women's 50 metre rifle three positions")]
[('sname', 'Pfälzer Turnerbund'), ('o', 'http://www.wikidata.org/entity/Q102483105'), ('oname', 'Deutschland-Cup Gerätturnen 2019')]
[('sname', 'Pfälzer Turnerbund'), ('o', 'http://www.wikidata.org/entity/Q103830609'), (

In [31]:
# Q1053559 --> '2020 Winter Youth Olympics'
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q1053559 ?p ?o .
   # get the label
   #?s sc:name ?sname .
   ?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q8204777'), ('oname', 'Category:2020 Winter Youth Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows'), ('o', 'http://www.wikidata.org/entity/Q370745'), ('oname', '2016 Winter Youth Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q74126424'), ('oname', 'Ice hockey at the 2020 Winter Youth Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('o', 'http://www.wikidata.org/entity/Q166788'), ('oname', 'biathlon')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('o', 'http://www.wikidata.org/entity/Q177275'), ('oname', 'bobsleigh')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('o', 'http://www.wikidata.org/entity/Q273285'), ('oname', 'luge')]
[('p', 'http://www.wikidat

In [20]:
# P910 --> "topic's main category"
# Q1053559 --> '2020 Winter Youth Olympics'
queryString = """
SELECT ?sname ?oname
WHERE {
   # bind something
   ?s wdt:P910 ?o .
   # get the label
   ?s sc:name ?sname .
   #?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('sname', 'Adelina Lopes Vieira'), ('oname', 'Category:Adelina Lopes Vieira')]
[('sname', 'Teluk Sebong'), ('oname', 'Category:Teluk Sebong')]
[('sname', 'Water supply and sanitation in Nicaragua'), ('oname', 'Category:Water supply and sanitation in Nicaragua')]
[('sname', 'Aganosma'), ('oname', 'Category:Aganosma')]
[('sname', 'Axyris'), ('oname', 'Category:Axyris')]
[('sname', 'Actinoscirpus'), ('oname', 'Category:Actinoscirpus')]
[('sname', 'Actinostemma'), ('oname', 'Category:Actinostemma')]
[('sname', 'Agrostocrinum'), ('oname', 'Category:Agrostocrinum')]
[('sname', 'Allomaieta'), ('oname', 'Category:Allomaieta')]
[('sname', 'Amphilophium'), ('oname', 'Category:Amphilophium')]
[('sname', 'Ascolepis'), ('oname', 'Category:Ascolepis')]
[('sname', 'Asta'), ('oname', 'Category:Asta')]
[('sname', 'Asteropyrum'), ('oname', 'Category:Asteropyrum')]
[('sname', 'Valladolid City Council'), ('oname', 'Category:City Council of Valladolid')]
[('sname', 'Ancylostemon'), ('oname', 'Cate

In [21]:
# P31 --> 'instance of'
# Q1053559 --> '2020 Winter Youth Olympics'
queryString = """
SELECT ?sname ?oname
WHERE {
   # bind something
   ?s wdt:P31 ?o .
   # get the label
   ?s sc:name ?sname .
   #?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('sname', 'ambassador of Germany to South Korea'), ('oname', 'ambassador to South Korea')]
[('sname', 'ambassador of Canada to Laos'), ('oname', 'ambassador of Canada')]
[('sname', 'ambassador of Canada to Haiti'), ('oname', 'ambassador of Canada')]
[('sname', 'ambassador of Canada to Guinea-Bissau'), ('oname', 'ambassador of Canada')]
[('sname', 'ambassador of Canada to Iraq'), ('oname', 'ambassador of Canada')]
[('sname', 'ambassador of Canada to Vietnam'), ('oname', 'ambassador of Canada')]
[('sname', 'ambassador of Canada to Yugoslavia'), ('oname', 'ambassador of Canada')]
[('sname', 'ambassador of Canada to Bulgaria'), ('oname', 'ambassador of Canada')]
[('sname', 'ambassador of Canada to the Comoros'), ('oname', 'ambassador of Canada')]
[('sname', 'ambassador of Canada to Rwanda'), ('oname', 'ambassador of Canada')]
[('sname', 'ambassador of Canada to the Democratic Republic of the Congo'), ('oname', 'ambassador of Canada')]
[('sname', 'ambassador of Canada to Gabon'), (

In [24]:
# P155 --> 'follows'
# Q1053559 --> '2020 Winter Youth Olympics'
queryString = """
SELECT ?sname ?oname
WHERE {
   # bind something
   ?s wdt:P155 ?o .
   # get the label
   ?s sc:name ?sname .
   #?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('sname', 'Category:1921 elections in Oceania'), ('oname', 'Category:1920 elections in Oceania')]
[('sname', '1995 in sports'), ('oname', '1994 in sports')]
[('sname', '1987 in sports'), ('oname', '1986 in sports')]
[('sname', '2039'), ('oname', '2038')]
[('sname', '2038'), ('oname', '2037')]
[('sname', '2000 in the Republic of Macedonia'), ('oname', '1999 in the Republic of Macedonia')]
[('sname', '1933 in Latvia'), ('oname', '1932 in Latvia')]
[('sname', 'Category:1927 in Argentine sport'), ('oname', 'Category:1926 in Argentine sport')]
[('sname', 'Category:1929 in South American sport'), ('oname', 'Category:1928 in South American sport')]
[('sname', 'Category:1945 in American sports'), ('oname', 'Category:1944 in American sports')]
[('sname', 'Category:1920 in Hawaii'), ('oname', 'Category:1919 in Hawaii')]
[('sname', 'Category:1935 in American sports'), ('oname', 'Category:1934 in American sports')]
[('sname', 'Category:1939 in Algeria'), ('oname', 'Category:1938 in Algeri

In [29]:
# Q1053559 --> '2020 Winter Youth Olympics'
queryString = """
SELECT ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q1053559 ?p ?o .
   # get the label
   #?s sc:name ?sname .
   ?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q8204777'), ('oname', 'Category:2020 Winter Youth Olympics')]
[('pname', 'follows'), ('o', 'http://www.wikidata.org/entity/Q370745'), ('oname', '2016 Winter Youth Olympics')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q74126424'), ('oname', 'Ice hockey at the 2020 Winter Youth Olympics')]
[('pname', 'sport'), ('o', 'http://www.wikidata.org/entity/Q166788'), ('oname', 'biathlon')]
[('pname', 'sport'), ('o', 'http://www.wikidata.org/entity/Q177275'), ('oname', 'bobsleigh')]
[('pname', 'sport'), ('o', 'http://www.wikidata.org/entity/Q273285'), ('oname', 'luge')]
[('pname', 'sport'), ('o', 'http://www.wikidata.org/entity/Q1075998'), ('oname', 'ski mountaineering')]
[('pname', 'sport'), ('o', 'http://www.wikidata.org/entity/Q189472'), ('oname', 'freestyle skiing')]
[('pname', 'sport'), ('o', 'http://www.wikidata.org/entity/Q193654'), ('oname', 'short track speed skating')]
[('pname', 'sport

In [30]:
# Q8204777' --> 'Category:2020 Winter Youth Olympics'
queryString = """
SELECT ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q8204777 ?p ?o .
   # get the label
   #?s sc:name ?sname .
   ?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('pname', "category's main topic"), ('o', 'http://www.wikidata.org/entity/Q1053559'), ('oname', '2020 Winter Youth Olympics')]
[('pname', 'category combines topics'), ('o', 'http://www.wikidata.org/entity/Q25337'), ('oname', '2020')]
[('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q4167836'), ('oname', 'Wikimedia category')]
3


In [32]:
# Q3178414 --> 'Winter Youth Olympic Games'
queryString = """
SELECT ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q3178414 ?p ?o .
   # get the label
   #?s sc:name ?sname .
   ?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q9530266'), ('oname', 'Category:Winter Youth Olympic Games')]
[('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q167170'), ('oname', 'multi-sport event')]
[('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q18608583'), ('oname', 'recurring sporting event')]
[('pname', 'sport'), ('o', 'http://www.wikidata.org/entity/Q204686'), ('oname', 'winter sport')]
[('pname', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q40970'), ('oname', 'International Olympic Committee')]
[('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q186196'), ('oname', 'Youth Olympic Games')]
[('pname', 'opposite of'), ('o', 'http://www.wikidata.org/entity/Q3178415'), ('oname', 'Summer Youth Olympic Games')]
7


In [34]:
# Q40970 --> 'International Olympic Committee'
queryString = """
SELECT ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q40970 ?p ?o .
   # get the label
   #?s sc:name ?sname .
   ?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pname', 'subsidiary'), ('o', 'http://www.wikidata.org/entity/Q282595'), ('oname', 'Olympic Broadcasting Services')]
[('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q11422536'), ('oname', 'international sport governing body')]
[('pname', 'office held by head of the organization'), ('o', 'http://www.wikidata.org/entity/Q7241513'), ('oname', 'president of the International Olympic Committee')]
[('pname', 'subsidiary'), ('o', 'http://www.wikidata.org/entity/Q1199476'), ('oname', 'Panam Sports')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q17152513'), ('oname', "IOC Athletes' Commission")]
[('pname', 'official language'), ('o', 'http://www.wikidata.org/entity/Q150'), ('oname', 'French')]
[('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q163740'), ('oname', 'nonprofit organization')]
[('pname', 'official language'), ('o', 'http://www.wikidata.org/entity/Q1860'), ('oname', 'English')]
[('pname', 'operating area'), ('o', 'http://ww

In [37]:
# Q5389 --> 'Olympic Games'
queryString = """
SELECT ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q5389 ?p ?o .
   # get the label
   #?s sc:name ?sname .
   ?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('pname', 'motto'), ('o', 'http://www.wikidata.org/entity/Q1093409'), ('oname', 'Citius, Altius, Fortius')]
[('pname', 'flag'), ('o', 'http://www.wikidata.org/entity/Q14624058'), ('oname', 'Olympic flag')]
[('pname', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q159821'), ('oname', 'Summer Olympic Games')]
[('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q167170'), ('oname', 'multi-sport event')]
[('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q18608583'), ('oname', 'recurring sporting event')]
[('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q18608583'), ('oname', 'recurring sporting event')]
[('pname', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q19180675'), ('oname', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('pname', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q19190511'), ('oname', 'New Encyclopedic Dictionary')]
[('pname', "topic's main category"), ('o', 

Final query for this task

In [38]:
# write your final query
# Q5389 --> 'Olympic Games'
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q5389 ?p ?o .
   # get the label
   #?s sc:name ?sname .
   ?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1546'), ('pname', 'motto'), ('o', 'http://www.wikidata.org/entity/Q1093409'), ('oname', 'Citius, Altius, Fortius')]
[('p', 'http://www.wikidata.org/prop/direct/P163'), ('pname', 'flag'), ('o', 'http://www.wikidata.org/entity/Q14624058'), ('oname', 'Olympic flag')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pname', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q159821'), ('oname', 'Summer Olympic Games')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q167170'), ('oname', 'multi-sport event')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q18608583'), ('oname', 'recurring sporting event')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q18608583'), ('oname', 'recurring sporting event')]
[('p', 'http:/

## Task 2
Return all the editions of the Summer Olympic Games (do not consider future Olympic Games) with the country where they were played (the result set must be a list of elements with edition IRI and label, and country IRI and label).

In [44]:
# write your queries
# Q159821 --> 'Summer Olympic Games'
queryString = """
SELECT ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q159821 ?p ?o .
   # get the label
   #?s sc:name ?sname .
   ?p sc:name ?pname.
   ?o sc:name ?oname
}
ORDER BY (?oname)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q8080'), ('oname', '1896 Summer Olympics')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q8088'), ('oname', '1900 Summer Olympics')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q8098'), ('oname', '1904 Summer Olympics')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q8105'), ('oname', '1906 Intercalated Games')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q8111'), ('oname', '1908 Summer Olympics')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q8118'), ('oname', '1912 Summer Olympics')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q8123'), ('oname', '1916 Summer Olympics')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q8128'), ('oname', '1920 Summer Olympics')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q8132'), ('oname', '1924 Summer Olympics')]
[('pname', 'has part'), ('o', 'http

In [46]:
# Q31039462 --> '2032 Summer Olympics'
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q31039462 ?p ?o .
   # get the label
   #?s sc:name ?sname .
   ?p sc:name ?pname.
   ?o sc:name ?oname
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q159821'), ('oname', 'Summer Olympic Games')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q408'), ('oname', 'Australia')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location'), ('o', 'http://www.wikidata.org/entity/Q34932'), ('oname', 'Brisbane')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows'), ('o', 'http://www.wikidata.org/entity/Q1451505'), ('oname', '2028 Summer Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q32699055'), ('oname', 'Category:2032 Summer Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pname', 'followed by'), ('o', 'http://www.wikidata.org/entity/Q108045332'), ('oname', '2036 Summer Olympics')]
6


In [45]:
#removing by oreder by and limit the future olympic games and not relevant nodes
# Q159821 --> 'Summer Olympic Games'
queryString = """
SELECT ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q159821 ?p ?o .
   # get the label
   #?s sc:name ?sname .
   ?p sc:name ?pname.
   ?o sc:name ?oname
}

ORDER BY (?oname)
LIMIT 33
"""

print("Results")
x=run_query(queryString)

Results
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q8080'), ('oname', '1896 Summer Olympics')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q8088'), ('oname', '1900 Summer Olympics')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q8098'), ('oname', '1904 Summer Olympics')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q8105'), ('oname', '1906 Intercalated Games')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q8111'), ('oname', '1908 Summer Olympics')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q8118'), ('oname', '1912 Summer Olympics')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q8123'), ('oname', '1916 Summer Olympics')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q8128'), ('oname', '1920 Summer Olympics')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q8132'), ('oname', '1924 Summer Olympics')]
[('pname', 'has part'), ('o', 'http

In [64]:
#binding countries too
# P17 --> 'country'
# Q159821 --> 'Summer Olympic Games'
queryString = """
SELECT  ?SOGname ?countryname ?anyOlympicGame ?country  
WHERE {
   # bind something
   wd:Q159821 ?haspart ?anyOlympicGame .
   ?anyOlympicGame wdt:P17 ?country .
   # get the label
   #?s sc:name ?sname .
   #?p sc:name ?pname.
   ?country sc:name ?countryname .
   ?anyOlympicGame sc:name ?SOGname
}

ORDER BY (?SOGname)
LIMIT 33
"""

print("Results")
x=run_query(queryString)

Results
[('SOGname', '1896 Summer Olympics'), ('countryname', 'Kingdom of Greece'), ('anyOlympicGame', 'http://www.wikidata.org/entity/Q8080'), ('country', 'http://www.wikidata.org/entity/Q209065')]
[('SOGname', '1900 Summer Olympics'), ('countryname', 'France'), ('anyOlympicGame', 'http://www.wikidata.org/entity/Q8088'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('SOGname', '1904 Summer Olympics'), ('countryname', 'United States of America'), ('anyOlympicGame', 'http://www.wikidata.org/entity/Q8098'), ('country', 'http://www.wikidata.org/entity/Q30')]
[('SOGname', '1906 Intercalated Games'), ('countryname', 'Kingdom of Greece'), ('anyOlympicGame', 'http://www.wikidata.org/entity/Q8105'), ('country', 'http://www.wikidata.org/entity/Q209065')]
[('SOGname', '1908 Summer Olympics'), ('countryname', 'United Kingdom of Great Britain and Ireland'), ('anyOlympicGame', 'http://www.wikidata.org/entity/Q8111'), ('country', 'http://www.wikidata.org/entity/Q174193')]
[('SOGname', '1912 

Final query for this task

In [65]:
# write your final query
# P17 --> 'country'
# Q159821 --> 'Summer Olympic Games'
queryString = """
SELECT  ?SOGname ?countryname ?anyOlympicGame ?country  
WHERE {
   # bind something
   wd:Q159821 ?haspart ?anyOlympicGame .
   ?anyOlympicGame wdt:P17 ?country .
   # get the label
   ?country sc:name ?countryname .
   ?anyOlympicGame sc:name ?SOGname
}

ORDER BY (?SOGname)
LIMIT 33
"""

print("Results")
x=run_query(queryString)

Results
[('SOGname', '1896 Summer Olympics'), ('countryname', 'Kingdom of Greece'), ('anyOlympicGame', 'http://www.wikidata.org/entity/Q8080'), ('country', 'http://www.wikidata.org/entity/Q209065')]
[('SOGname', '1900 Summer Olympics'), ('countryname', 'France'), ('anyOlympicGame', 'http://www.wikidata.org/entity/Q8088'), ('country', 'http://www.wikidata.org/entity/Q142')]
[('SOGname', '1904 Summer Olympics'), ('countryname', 'United States of America'), ('anyOlympicGame', 'http://www.wikidata.org/entity/Q8098'), ('country', 'http://www.wikidata.org/entity/Q30')]
[('SOGname', '1906 Intercalated Games'), ('countryname', 'Kingdom of Greece'), ('anyOlympicGame', 'http://www.wikidata.org/entity/Q8105'), ('country', 'http://www.wikidata.org/entity/Q209065')]
[('SOGname', '1908 Summer Olympics'), ('countryname', 'United Kingdom of Great Britain and Ireland'), ('anyOlympicGame', 'http://www.wikidata.org/entity/Q8111'), ('country', 'http://www.wikidata.org/entity/Q174193')]
[('SOGname', '1912 

## Task 3
Find the countries which held more than 2 Olympic Games (both Summer and Winter Games, do not consider future Olympic Games) (the result set must be a list of triples country IRI, label and #edition held).

In [70]:
# write your queries
# Q82414 --> 'Olympic Games' took from task2
queryString = """
SELECT  ?p ?o ?pname ?oname
WHERE {
   # bind something
   wd:Q82414 ?p ?o .
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname
}

ORDER BY (?oname)
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('o', 'http://www.wikidata.org/entity/Q9583'), ('pname', 'has part'), ('oname', '1924 Winter Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('o', 'http://www.wikidata.org/entity/Q9591'), ('pname', 'has part'), ('oname', '1928 Winter Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('o', 'http://www.wikidata.org/entity/Q9596'), ('pname', 'has part'), ('oname', '1932 Winter Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('o', 'http://www.wikidata.org/entity/Q9600'), ('pname', 'has part'), ('oname', '1936 Winter Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('o', 'http://www.wikidata.org/entity/Q9607'), ('pname', 'has part'), ('oname', '1940 Winter Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('o', 'http://www.wikidata.org/entity/Q9613'), ('pname', 'has part'), ('oname', '1944 Winter Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P527')

In [71]:
# write your queries
# Q82414 --> 'Winter Olympic Games' took from the task2
# P17 --> 'country'
# Q159821 --> 'Summer Olympic Games'
# Q5389 --> 'Olympic Games'
queryString = """
SELECT  ?p ?o ?pname ?oname
WHERE {
   # bind something
   wd:Q5389 ?p ?o .
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname
}

ORDER BY (?oname)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('o', 'http://www.wikidata.org/entity/Q602358'), ('pname', 'described by source'), ('oname', 'Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('o', 'http://www.wikidata.org/entity/Q1983209'), ('pname', "topic's main category"), ('oname', 'Category:Olympic Games')]
[('p', 'http://www.wikidata.org/prop/direct/P1546'), ('o', 'http://www.wikidata.org/entity/Q1093409'), ('pname', 'motto'), ('oname', 'Citius, Altius, Fortius')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('o', 'http://www.wikidata.org/entity/Q40970'), ('pname', 'organizer'), ('oname', 'International Olympic Committee')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('o', 'http://www.wikidata.org/entity/Q19190511'), ('pname', 'described by source'), ('oname', 'New Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P163'), ('o', 'http://www.wikidata.org/entity/Q14624058'), ('pname',

In [72]:
# P2670 --> 'has parts of the class'
# Q82414 --> 'Winter Olympic Games'
# P17 --> 'country'
# Q159821 --> 'Summer Olympic Games'
# Q5389 --> 'Olympic Games'
queryString = """
SELECT  ?OGname
WHERE {
   # bind something
   wd:Q5389 wdt:P2670 ?OG .
   # get the label
   #?p sc:name ?pname .
   ?OG sc:name ?OGname
}

ORDER BY (?OGname)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('OGname', 'Summer Olympic Games')]
[('OGname', 'Winter Olympic Games')]
2


In [77]:
# P2670 --> 'has parts of the class'
# Q82414 --> 'Winter Olympic Games'
# P17 --> 'country'
# Q159821 --> 'Summer Olympic Games'
# Q5389 --> 'Olympic Games'
queryString = """
SELECT  ?OGname ?allOGname
WHERE {
   # bind something
   wd:Q5389 wdt:P2670 ?OG .
   ?OG ?p ?allOG .
   # get the label
   #?p sc:name ?pname .
   ?OG sc:name ?OGname .
   ?allOG sc:name ?allOGname
}

ORDER BY (?allOGname)
LIMIT 59
"""

print("Results")
x=run_query(queryString)

Results
[('OGname', 'Summer Olympic Games'), ('allOGname', '1896 Summer Olympics')]
[('OGname', 'Summer Olympic Games'), ('allOGname', '1900 Summer Olympics')]
[('OGname', 'Summer Olympic Games'), ('allOGname', '1904 Summer Olympics')]
[('OGname', 'Summer Olympic Games'), ('allOGname', '1906 Intercalated Games')]
[('OGname', 'Summer Olympic Games'), ('allOGname', '1908 Summer Olympics')]
[('OGname', 'Summer Olympic Games'), ('allOGname', '1912 Summer Olympics')]
[('OGname', 'Summer Olympic Games'), ('allOGname', '1916 Summer Olympics')]
[('OGname', 'Summer Olympic Games'), ('allOGname', '1920 Summer Olympics')]
[('OGname', 'Summer Olympic Games'), ('allOGname', '1924 Summer Olympics')]
[('OGname', 'Winter Olympic Games'), ('allOGname', '1924 Winter Olympics')]
[('OGname', 'Summer Olympic Games'), ('allOGname', '1928 Summer Olympics')]
[('OGname', 'Winter Olympic Games'), ('allOGname', '1928 Winter Olympics')]
[('OGname', 'Summer Olympic Games'), ('allOGname', '1932 Summer Olympics')]
[

In [99]:
# P2670 --> 'has parts of the class'
# Q82414 --> 'Winter Olympic Games'
# P17 --> 'country'
# Q159821 --> 'Summer Olympic Games'
# Q5389 --> 'Olympic Games'
queryString = """
SELECT  ?OGname ?allOG ?allOGname ?coutryname
WHERE {
   # bind something
   wd:Q5389 wdt:P2670 ?OG .
   ?OG ?p ?allOG .
   ?allOG wdt:P17 ?country .
   # get the label
   #?p sc:name ?pname .
   ?OG sc:name ?OGname .
   ?allOG sc:name ?allOGname .
   ?country sc:name ?coutryname
}

ORDER BY DESC(?allOGname)
#Having (2024 Summer)
LIMIT 70
"""

print("Results")
x=run_query(queryString)

Results
[('OGname', 'Summer Olympic Games'), ('allOG', 'http://www.wikidata.org/entity/Q99413897'), ('allOGname', 'Medvik'), ('coutryname', 'Czech Republic')]
[('OGname', 'Summer Olympic Games'), ('allOG', 'http://www.wikidata.org/entity/Q40970'), ('allOGname', 'International Olympic Committee'), ('coutryname', 'Switzerland')]
[('OGname', 'Winter Olympic Games'), ('allOG', 'http://www.wikidata.org/entity/Q40970'), ('allOGname', 'International Olympic Committee'), ('coutryname', 'Switzerland')]
[('OGname', 'Summer Olympic Games'), ('allOG', 'http://www.wikidata.org/entity/Q31039462'), ('allOGname', '2032 Summer Olympics'), ('coutryname', 'Australia')]
[('OGname', 'Summer Olympic Games'), ('allOG', 'http://www.wikidata.org/entity/Q1451505'), ('allOGname', '2028 Summer Olympics'), ('coutryname', 'United States of America')]
[('OGname', 'Winter Olympic Games'), ('allOG', 'http://www.wikidata.org/entity/Q4630399'), ('allOGname', '2026 Winter Olympics'), ('coutryname', 'Italy')]
[('OGname', 

In [130]:
# Q193074 --> '2022 Winter Olympics'
# Q995653 --> '2024 Summer Olympics'
queryString = """
SELECT  ?OGname ?allOG ?allOGname ?coutryname
WHERE {
   # bind something
   wd:Q5389 wdt:P2670 ?OG .
   ?OG ?p ?allOG .
   ?allOG wdt:P17 ?country .
   # get the label
   #?p sc:name ?pname .
   ?OG sc:name ?OGname .
   ?allOG sc:name ?allOGname .
   ?country sc:name ?coutryname .
   MINUS{ wd:Q995653 sc:name ?allOGname.}  
   MINUS{ wd:Q4630399 sc:name ?allOGname. }
   MINUS{ wd:Q1451505 sc:name ?allOGname. }
   MINUS{ wd:Q31039462 sc:name ?allOGname. }
   MINUS{ wd:Q40970 sc:name ?allOGname. }
   MINUS{ wd:Q99413897 sc:name ?allOGname. }
   
   
}

ORDER BY DESC(?allOGname)
#OFFSET 7
LIMIT 70
"""

print("Results")
x=run_query(queryString)

Results
[('OGname', 'Winter Olympic Games'), ('allOG', 'http://www.wikidata.org/entity/Q193074'), ('allOGname', '2022 Winter Olympics'), ('coutryname', "People's Republic of China")]
[('OGname', 'Summer Olympic Games'), ('allOG', 'http://www.wikidata.org/entity/Q181278'), ('allOGname', '2020 Summer Olympics'), ('coutryname', 'Japan')]
[('OGname', 'Winter Olympic Games'), ('allOG', 'http://www.wikidata.org/entity/Q9680'), ('allOGname', '2018 Winter Olympics'), ('coutryname', 'South Korea')]
[('OGname', 'Summer Olympic Games'), ('allOG', 'http://www.wikidata.org/entity/Q8613'), ('allOGname', '2016 Summer Olympics'), ('coutryname', 'Brazil')]
[('OGname', 'Winter Olympic Games'), ('allOG', 'http://www.wikidata.org/entity/Q9678'), ('allOGname', '2014 Winter Olympics'), ('coutryname', 'Russia')]
[('OGname', 'Summer Olympic Games'), ('allOG', 'http://www.wikidata.org/entity/Q8577'), ('allOGname', '2012 Summer Olympics'), ('coutryname', 'United Kingdom')]
[('OGname', 'Winter Olympic Games'), (

In [144]:
# P2670 --> 'has parts of the class'
# Q82414 --> 'Winter Olympic Games'
# P17 --> 'country'
# Q159821 --> 'Summer Olympic Games'
# Q5389 --> 'Olympic Games'
queryString = """
SELECT   ?countryname (COUNT(?countryname) AS ?CountryCount) #?allOGname
WHERE {
   # bind something
   wd:Q5389 wdt:P2670 ?OG .
   ?OG ?p ?allOG .
   ?allOG wdt:P17 ?country .
   # get the label
   #?p sc:name ?pname .
   ?OG sc:name ?OGname .
   ?allOG sc:name ?allOGname .
   ?country sc:name ?countryname .
   MINUS{ wd:Q995653 sc:name ?allOGname.}  
   MINUS{ wd:Q4630399 sc:name ?allOGname. }
   MINUS{ wd:Q1451505 sc:name ?allOGname. }
   MINUS{ wd:Q31039462 sc:name ?allOGname. }
   MINUS{ wd:Q40970 sc:name ?allOGname. }
   MINUS{ wd:Q99413897 sc:name ?allOGname. }
}
GROUP BY ?CountryCount ?countryname
HAVING (COUNT(?country) > 1)
ORDER BY desc(?CountryCount) 
#OFFSET 6

#order by desc(?CountryCount)
LIMIT 60
"""

print("Results")
x=run_query(queryString)

Results
[('countryname', 'United States of America'), ('CountryCount', '7')]
[('countryname', 'Japan'), ('CountryCount', '6')]
[('countryname', 'France'), ('CountryCount', '5')]
[('countryname', 'German Reich'), ('CountryCount', '3')]
[('countryname', 'Canada'), ('CountryCount', '3')]
[('countryname', 'United Kingdom'), ('CountryCount', '3')]
[('countryname', "People's Republic of China"), ('CountryCount', '2')]
[('countryname', 'Mexico'), ('CountryCount', '2')]
[('countryname', 'Australia'), ('CountryCount', '2')]
[('countryname', 'Italy'), ('CountryCount', '2')]
[('countryname', 'Kingdom of Greece'), ('CountryCount', '2')]
[('countryname', 'Switzerland'), ('CountryCount', '2')]
[('countryname', 'Brazil'), ('CountryCount', '2')]
[('countryname', 'South Korea'), ('CountryCount', '2')]
[('countryname', 'Austria'), ('CountryCount', '2')]
[('countryname', 'Norway'), ('CountryCount', '2')]
16


Final query for this task

In [146]:
# write your final query
# P2670 --> 'has parts of the class'
# Q82414 --> 'Winter Olympic Games'
# P17 --> 'country'
# Q159821 --> 'Summer Olympic Games'
# Q5389 --> 'Olympic Games'
queryString = """
SELECT   ?country ?countryname (COUNT(?countryname) AS ?CountryCount) #?allOGname
WHERE {
   # bind something
   wd:Q5389 wdt:P2670 ?OG .
   ?OG ?p ?allOG .
   ?allOG wdt:P17 ?country .
   # get the label
   ?OG sc:name ?OGname .
   ?allOG sc:name ?allOGname .
   ?country sc:name ?countryname .
   #excluding unrelevant nodes
   MINUS{ wd:Q995653 sc:name ?allOGname.}  
   MINUS{ wd:Q4630399 sc:name ?allOGname. }
   MINUS{ wd:Q1451505 sc:name ?allOGname. }
   MINUS{ wd:Q31039462 sc:name ?allOGname. }
   MINUS{ wd:Q40970 sc:name ?allOGname. }
   MINUS{ wd:Q99413897 sc:name ?allOGname. }
}
GROUP BY ?country ?CountryCount ?countryname
HAVING (COUNT(?country) > 1)
ORDER BY desc(?CountryCount)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q30'), ('countryname', 'United States of America'), ('CountryCount', '7')]
[('country', 'http://www.wikidata.org/entity/Q17'), ('countryname', 'Japan'), ('CountryCount', '6')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryname', 'France'), ('CountryCount', '5')]
[('country', 'http://www.wikidata.org/entity/Q1206012'), ('countryname', 'German Reich'), ('CountryCount', '3')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryname', 'United Kingdom'), ('CountryCount', '3')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('countryname', 'Canada'), ('CountryCount', '3')]
[('country', 'http://www.wikidata.org/entity/Q148'), ('countryname', "People's Republic of China"), ('CountryCount', '2')]
[('country', 'http://www.wikidata.org/entity/Q39'), ('countryname', 'Switzerland'), ('CountryCount', '2')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryname', 'Italy'), ('CountryCount', '2')]
[('country'

## Task 4
Consider the 2008 Summer Olympic Games. Identify all the sports played in this edition (the result set must be a list of couples sport IRI and label).

In [152]:
# write your queries
# Q8567 --> '2008 Summer Olympics' took from task3
queryString = """
SELECT DISTINCT ?p ?pname ?oname
WHERE {
   # bind something
   wd:Q8567 ?p ?o .
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname
 }

LIMIT 100
"""

print("Results")
x=run_query(queryString)  

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('oname', 'field hockey at the 2008 Summer Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P542'), ('pname', 'officially opened by'), ('oname', 'Hu Jintao')]
[('p', 'http://www.wikidata.org/prop/direct/P543'), ('pname', 'oath made by'), ('oname', 'Zhang Yining')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('oname', 'table tennis at the 2008 Summer Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('oname', 'wrestling at the 2008 Summer Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('oname', 'swimming at the 2008 Summer Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('oname', 'water polo at the 2008 Summer Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('oname', 'weightlifting at the 2008 Summer Olympics')]
[('

In [153]:
# P527 --> 'has part'
# Q8567 --> '2008 Summer Olympics' took from task3
queryString = """
SELECT DISTINCT ?p ?pname ?oname
WHERE {
   # bind something
   wd:Q8567 wdt:P527 ?o .
   # get the label
   #?p sc:name ?pname .
   ?o sc:name ?oname
 }

LIMIT 100
"""

print("Results")
x=run_query(queryString) 

Results
[('oname', 'field hockey at the 2008 Summer Olympics')]
[('oname', 'table tennis at the 2008 Summer Olympics')]
[('oname', 'wrestling at the 2008 Summer Olympics')]
[('oname', 'swimming at the 2008 Summer Olympics')]
[('oname', 'water polo at the 2008 Summer Olympics')]
[('oname', 'weightlifting at the 2008 Summer Olympics')]
[('oname', 'boxing at the 2008 Summer Olympics')]
[('oname', 'basketball at the 2008 Summer Olympics')]
[('oname', 'trampoline gymnastics at the 2008 Summer Olympics')]
[('oname', 'badminton at the 2008 Summer Olympics')]
[('oname', 'artistic gymnastics at the 2008 Summer Olympics')]
[('oname', 'cycling at the 2008 Summer Olympics')]
[('oname', 'canoeing at the 2008 Summer Olympics')]
[('oname', 'shooting at the 2008 Summer Olympics')]
[('oname', 'diving at the 2008 Summer Olympics')]
[('oname', 'rhythmic gymnastics at the 2008 Summer Olympics')]
[('oname', 'tennis at the 2008 Summer Olympics')]
[('oname', 'football at the 2008 Summer Olympics')]
[('oname'

Final query for this task

In [154]:
# write your final query
# P527 --> 'has part'
# Q8567 --> '2008 Summer Olympics' took from task3
queryString = """
SELECT DISTINCT ?o ?oname
WHERE {
   # bind something
   wd:Q8567 wdt:P527 ?o .
   # get the label
   #?p sc:name ?pname .
   ?o sc:name ?oname
 }

LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q221132'), ('oname', 'field hockey at the 2008 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q387526'), ('oname', 'table tennis at the 2008 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q220654'), ('oname', 'wrestling at the 2008 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q204844'), ('oname', 'swimming at the 2008 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q220942'), ('oname', 'water polo at the 2008 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q212887'), ('oname', 'weightlifting at the 2008 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q220930'), ('oname', 'boxing at the 2008 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q189816'), ('oname', 'basketball at the 2008 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q2022469'), ('oname', 'trampoline gymnastics at the 2008 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q207004'), ('o

## Task 5
Consider the 2008 Summer Olympic Games. For each sport return the number of different disciplines (the result set must be a list of triples sport IRI, label and #disciplines).

In [159]:
# write your queries
# Q381115 --> 'taekwondo at the 2008 Summer Olympics'
# P527 --> 'has part'
# Q8567 --> '2008 Summer Olympics' 
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q381115 ?p ?o .
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname
 }

LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q1005783'), ('oname', "taekwondo at the 2008 Summer Olympics – men's 80 kg")]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q1005660'), ('oname', "taekwondo at the 2008 Summer Olympics – men's +80 kg")]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q148'), ('oname', "People's Republic of China")]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pname', 'followed by'), ('o', 'http://www.wikidata.org/entity/Q239676'), ('oname', 'taekwondo at the 2012 Summer Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q26132862'), ('oname', 'Olympic sports discipline event')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows'), ('o', 'http://www.wiki

In [157]:
# P527 --> 'has part'
# Q8567 --> '2008 Summer Olympics' 
# P641 --> 'sport'
queryString = """
SELECT DISTINCT ?sportname
WHERE {
   # bind something
   wd:Q8567 wdt:P527 ?fields .
   ?fields wdt:P641 ?sport . 
   # get the label
   #?p sc:name ?pname .
   ?sport sc:name ?sportname
 }

LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('sportname', 'triathlon')]
[('sportname', 'canoeing and kayaking')]
[('sportname', 'artistic gymnastics')]
[('sportname', 'synchronized swimming')]
[('sportname', 'trampolining')]
[('sportname', 'field hockey')]
[('sportname', 'volleyball')]
[('sportname', 'shooting sport')]
[('sportname', 'association football')]
[('sportname', 'swimming')]
[('sportname', 'boxing')]
[('sportname', 'beach volleyball')]
[('sportname', 'baseball')]
[('sportname', 'basketball')]
[('sportname', 'athletics')]
[('sportname', 'rhythmic gymnastics')]
[('sportname', 'water polo')]
[('sportname', 'tennis')]
[('sportname', 'judo')]
[('sportname', 'archery')]
[('sportname', 'fencing')]
[('sportname', 'sailing')]
[('sportname', 'rowing')]
[('sportname', 'cycle sport')]
[('sportname', 'modern pentathlon')]
[('sportname', 'table tennis')]
[('sportname', 'wrestling')]
[('sportname', 'gymnastics')]
[('sportname', 'diving')]
[('sportname', 'weightlifting')]
[('sportname', 'handball')]
[('sportname', 'equestria

In [160]:
# Q36389' --> 'taekwondo'
queryString = """
SELECT DISTINCT ?pname ?oname
WHERE {
   # bind something
   wd:Q36389 ?p ?o .
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname
 }

LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('pname', 'authority'), ('oname', 'World Taekwondo')]
[('pname', 'instance of'), ('oname', 'type of sport')]
[('pname', 'subclass of'), ('oname', 'martial arts')]
[('pname', 'country of origin'), ('oname', 'Korean Empire')]
[('pname', 'practiced by'), ('oname', 'taekwondo athlete')]
[('pname', "topic's main Wikimedia portal"), ('oname', 'Portal:Taekwondo')]
[('pname', "topic's main category"), ('oname', 'Category:Taekwondo')]
[('pname', 'different from'), ('oname', 'Taekwondo')]
8


In [13]:
# P527 --> 'has part'
# Q8567 --> '2008 Summer Olympics' 
# P641 --> 'sport'
queryString = """
SELECT  ?sportname ?disciplinesname 
WHERE {
   # bind something
   wd:Q8567 wdt:P527 ?fields .
   ?fields wdt:P641 ?sport . 
   ?fields wdt:P527 ?disciplines .
   # get the label
   #?p sc:name ?pname .
   ?disciplines sc:name ?disciplinesname .
   ?sport sc:name ?sportname .
   ?fields sc:name ?fieldsname
 }

LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('sportname', 'synchronized swimming'), ('disciplinesname', "synchronized swimming at the 2008 Summer Olympics – women's team")]
[('sportname', 'canoeing and kayaking'), ('disciplinesname', "canoeing at the 2008 Summer Olympics – women's slalom K-1")]
[('sportname', 'judo'), ('disciplinesname', "judo at the 2008 Summer Olympics – women's 57 kg")]
[('sportname', 'archery'), ('disciplinesname', "archery at the 2008 Summer Olympics – women's team")]
[('sportname', 'athletics'), ('disciplinesname', "athletics at the 2008 Summer Olympics – men's discus throw")]
[('sportname', 'athletics'), ('disciplinesname', "athletics at the 2008 Summer Olympics – men's pole vault")]
[('sportname', 'badminton'), ('disciplinesname', "badminton at the 2008 Summer Olympics – men's singles")]
[('sportname', 'shooting sport'), ('disciplinesname', "shooting at the 2008 Summer Olympics – men's 25 metre rapid fire pistol")]
[('sportname', 'equestrian sport'), ('disciplinesname', 'equestrian at the 2008 S

Final query for this task

In [14]:
# write your final query
# P527 --> 'has part'
# Q8567 --> '2008 Summer Olympics' 
# P641 --> 'sport'
queryString = """
SELECT distinct ?sport ?sportname (COUNT(?disciplinesname) AS ?NumDisciplines)
WHERE {
   # bind something
   wd:Q8567 wdt:P527 ?fields .
   ?fields wdt:P641 ?sport . 
   ?fields wdt:P527 ?disciplines .
   # get the label
   #?p sc:name ?pname .
   ?disciplines sc:name ?disciplinesname .
   ?sport sc:name ?sportname .
   ?fields sc:name ?fieldsname
 }
ORDER BY desc(?NumDisciplines)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('sport', 'http://www.wikidata.org/entity/Q542'), ('sportname', 'athletics'), ('NumDisciplines', '47')]
[('sport', 'http://www.wikidata.org/entity/Q31920'), ('sportname', 'swimming'), ('NumDisciplines', '34')]
[('sport', 'http://www.wikidata.org/entity/Q42486'), ('sportname', 'wrestling'), ('NumDisciplines', '18')]
[('sport', 'http://www.wikidata.org/entity/Q2215841'), ('sportname', 'cycle sport'), ('NumDisciplines', '18')]
[('sport', 'http://www.wikidata.org/entity/Q213934'), ('sportname', 'canoeing and kayaking'), ('NumDisciplines', '16')]
[('sport', 'http://www.wikidata.org/entity/Q83462'), ('sportname', 'weightlifting'), ('NumDisciplines', '15')]
[('sport', 'http://www.wikidata.org/entity/Q206989'), ('sportname', 'shooting sport'), ('NumDisciplines', '15')]
[('sport', 'http://www.wikidata.org/entity/Q159354'), ('sportname', 'rowing'), ('NumDisciplines', '14')]
[('sport', 'http://www.wikidata.org/entity/Q326827'), ('sportname', 'artistic gymnastics'), ('NumDisciplines', '14

## Task 6
Consider the 2008 Summer Olympic Games. Find the top-10 people who won more gold medals (the result set must be a list of triples athlete IRI, label and #gold medal).

In [17]:
# write your queries
# Q381115 --> 'taekwondo at the 2008 Summer Olympics'
# P527 --> 'has part'
# Q8567 --> '2008 Summer Olympics' 
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q8567 ?p ?o .
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname
 }

LIMIT 70
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q221132'), ('oname', 'field hockey at the 2008 Summer Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P542'), ('pname', 'officially opened by'), ('o', 'http://www.wikidata.org/entity/Q15029'), ('oname', 'Hu Jintao')]
[('p', 'http://www.wikidata.org/prop/direct/P543'), ('pname', 'oath made by'), ('o', 'http://www.wikidata.org/entity/Q197335'), ('oname', 'Zhang Yining')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q387526'), ('oname', 'table tennis at the 2008 Summer Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q220654'), ('oname', 'wrestling at the 2008 Summer Olympics')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q204844'), ('oname', 'swimming 

In [19]:
# write your queries
# P710 --> 'participant'
# Q8567 --> '2008 Summer Olympics' 
queryString = """
SELECT DISTINCT  ?o ?oname
WHERE {
   # bind something
   wd:Q8567 wdt:P710 ?o .
   # get the label
   #?p sc:name ?pname .
   ?o sc:name ?oname
 }
ORDER BY DESC (?oname)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q7820570'), ('oname', 'Tomáš Fučík')]
[('o', 'http://www.wikidata.org/entity/Q140734'), ('oname', 'Saint Kitts and Nevis at the 2008 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q690324'), ('oname', 'Petr Svoboda')]
[('o', 'http://www.wikidata.org/entity/Q142047'), ('oname', 'Palau at the 2008 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q142033'), ('oname', 'Nauru at the 2008 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q141797'), ('oname', 'Malta at the 2008 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q141811'), ('oname', 'Liechtenstein at the 2008 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q140737'), ('oname', 'Kiribati at the 2008 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q230865'), ('oname', 'Homare Sawa')]
[('o', 'http://www.wikidata.org/entity/Q441135'), ('oname', 'Formiga')]
[('o', 'http://www.wikidata.org/entity/Q4943862'), ('oname', 'Fabiana da Si

In [20]:
# Q445344 --> 'Carli Lloyd'
queryString = """
SELECT DISTINCT  ?pname ?oname
WHERE {
   # bind something
   wd:Q445344 ?p ?o .
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname
 }
ORDER BY DESC (?oname)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('pname', 'position played on team / speciality'), ('oname', 'midfielder')]
[('pname', 'instance of'), ('oname', 'human')]
[('pname', 'participant in'), ('oname', "football at the 2020 Summer Olympics – women's tournament")]
[('pname', 'participant in'), ('oname', "football at the 2016 Summer Olympics – women's tournament")]
[('pname', 'participant in'), ('oname', 'football at the 2016 Summer Olympics')]
[('pname', 'participant in'), ('oname', "football at the 2012 Summer Olympics – women's tournament")]
[('pname', 'participant in'), ('oname', "football at the 2008 Summer Olympics – women's tournament")]
[('pname', 'sex or gender'), ('oname', 'female')]
[('pname', 'occupation'), ('oname', 'association football player')]
[('pname', 'sport'), ('oname', 'association football')]
[('pname', 'member of sports team'), ('oname', 'Western New York Flash')]
[('pname', 'member of sports team'), ('oname', "United States women's national under-23 soccer team")]
[('pname', 'member of sports

In [3]:
# I'll try to find all the medalists for whole Olympic Games first and only then sort them somehow 
# Q159821 --> 'Summer Olympic Games'
queryString = """
SELECT DISTINCT  ?o ?oname
WHERE {
   # bind something
   wd:Q159821 ?p ?o .
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname
 }
#ORDER BY DESC (?oname)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q8098'), ('oname', '1904 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8150'), ('oname', '1936 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8415'), ('oname', '1960 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q181278'), ('oname', '2020 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q18608583'), ('oname', 'recurring sporting event')]
[('o', 'http://www.wikidata.org/entity/Q40970'), ('oname', 'International Olympic Committee')]
[('o', 'http://www.wikidata.org/entity/Q5389'), ('oname', 'Olympic Games')]
[('o', 'http://www.wikidata.org/entity/Q8088'), ('oname', '1900 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q82414'), ('oname', 'Winter Olympic Games')]
[('o', 'http://www.wikidata.org/entity/Q8438'), ('oname', '1972 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8444'), ('oname', '1976 Summer Olympics')]
[('o', 'http://www.wikidata.org/entity/Q8470'), ('oname', '1988

In [6]:
# Q5389 --> 'Olympic Games'
queryString = """
SELECT DISTINCT  ?p ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q5389 ?p ?o .
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname
 }
#ORDER BY DESC (?oname)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1546'), ('pname', 'motto'), ('o', 'http://www.wikidata.org/entity/Q1093409'), ('oname', 'Citius, Altius, Fortius')]
[('p', 'http://www.wikidata.org/prop/direct/P163'), ('pname', 'flag'), ('o', 'http://www.wikidata.org/entity/Q14624058'), ('oname', 'Olympic flag')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pname', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q159821'), ('oname', 'Summer Olympic Games')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q167170'), ('oname', 'multi-sport event')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q18608583'), ('oname', 'recurring sporting event')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q18608583'), ('oname', 'recurring sporting event')]
[('p', 'http:/

In [8]:
# P4622 --> 'trophy awarded'
# Q636830 --> 'Olympic medal'
queryString = """
SELECT DISTINCT  ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q636830 ?p ?o .
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname
 }
#ORDER BY DESC (?oname)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('pname', 'conferred by'), ('o', 'http://www.wikidata.org/entity/Q40970'), ('oname', 'International Olympic Committee')]
[('pname', 'facet of'), ('o', 'http://www.wikidata.org/entity/Q5389'), ('oname', 'Olympic Games')]
[('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q131647'), ('oname', 'medal')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q15243387'), ('oname', 'Olympic gold medal')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q15889641'), ('oname', 'Olympic silver medal')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q15889643'), ('oname', 'Olympic bronze medal')]
[('pname', 'category for recipients of this award'), ('o', 'http://www.wikidata.org/entity/Q6307826'), ('oname', 'Category:Olympic medalists')]
[('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q7205677'), ('oname', 'Category:Olympic medals')]
[('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q12042581'), 

In [10]:
# Q15243387 --> 'Olympic gold medal'
queryString = """
SELECT DISTINCT  ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q15243387 ?p ?o .
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname
 }
#ORDER BY DESC (?oname)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q406039'), ('oname', 'gold medal')]
[('pname', 'made from material'), ('o', 'http://www.wikidata.org/entity/Q1090'), ('oname', 'silver')]
[('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q636830'), ('oname', 'Olympic medal')]
[('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q636830'), ('oname', 'Olympic medal')]
[('pname', 'made from material'), ('o', 'http://www.wikidata.org/entity/Q897'), ('oname', 'gold')]
[('pname', 'next lower rank'), ('o', 'http://www.wikidata.org/entity/Q15889641'), ('oname', 'Olympic silver medal')]
[('pname', 'opposite of'), ('o', 'http://www.wikidata.org/entity/Q15243424'), ('oname', 'Paralympic gold medal')]
[('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q12042581'), ('oname', 'Olympic award')]
8


In [46]:
# Q15243387 --> 'Olympic gold medal'
# Q406039 --> 'gold medal'
queryString = """
SELECT DISTINCT  ?p ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q406039 ?p ?o .
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname
 }
#ORDER BY DESC (?oname)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q57082336'), ('oname', 'award medal')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q618779'), ('oname', 'award')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q18740852'), ('oname', 'Category:Gold medals')]
[('p', 'http://www.wikidata.org/prop/direct/P1536'), ('pname', 'immediate cause of'), ('o', 'http://www.wikidata.org/entity/Q36593383'), ('oname', 'sports victory')]
[('p', 'http://www.wikidata.org/prop/direct/P3729'), ('pname', 'next lower rank'), ('o', 'http://www.wikidata.org/entity/Q847956'), ('oname', 'silver medal')]
5


In [44]:
# Q15243387 --> 'Olympic gold medal'
# Q406039 --> 'gold medal'
# Q36593383 --> 'sports victory'
queryString = """
SELECT DISTINCT  ?p ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q36593383 ?p ?o .
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname
 }
#ORDER BY DESC (?oname)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q18560095'), ('oname', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1478'), ('pname', 'has immediate cause'), ('o', 'http://www.wikidata.org/entity/Q406039'), ('oname', 'gold medal')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q50000'), ('oname', 'victory')]
[('p', 'http://www.wikidata.org/prop/direct/P1536'), ('pname', 'immediate cause of'), ('o', 'http://www.wikidata.org/entity/Q1366722'), ('oname', 'final')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q9568568'), ('oname', 'Category:Sports victory')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pname', 'followed by'), ('o', 'http://www.wikidata.org/entity/Q3275569'), ('oname', 'runner-up')]
6


In [34]:
# Q15243387 --> 'Olympic gold medal'
# Q406039 --> 'gold medal'
# Q36593383 --> 'sports victory'
# Q18560095 --> 'winner'
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q18560095 ?p ?o .
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname
 }
#ORDER BY DESC (?oname)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q215627'), ('oname', 'person')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q216353'), ('oname', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q4120621'), ('oname', 'rank')]
[('p', 'http://www.wikidata.org/prop/direct/P461'), ('pname', 'opposite of'), ('o', 'http://www.wikidata.org/entity/Q20861252'), ('oname', 'loser')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from'), ('o', 'http://www.wikidata.org/entity/Q97453439'), ('oname', 'awardee')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q4897819'), ('oname', 'role')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikid

In [38]:
# Q15243387 --> 'Olympic gold medal'
# Q406039 --> 'gold medal'
# Q36593383 --> 'sports victory'
# Q18560095 --> 'winner'
# P1687 --> 'Wikidata property' ; P1346 --> 'winner'
queryString = """
SELECT DISTINCT  ?sname ?oname 
WHERE {
   # bind something
   ?s wdt:P1346 ?o .
   # get the label
   ?s sc:name ?sname .
   #?p sc:name ?pname .
   ?o sc:name ?oname .
   
 }
#ORDER BY DESC (?oname)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('sname', 'Miss Colombia 2007'), ('oname', 'Taliana Vargas')]
[('sname', "2013 European Diving Championships – men's 1 metre springboard"), ('oname', 'Illya Kvasha')]
[('sname', '2014 Tropicale Amissa Bongo'), ('oname', 'Egoitz García')]
[('sname', '2014 Tour of Qatar'), ('oname', 'Guillaume Van Keirsbulck')]
[('sname', '2014 WGC-Accenture Match Play Championship'), ('oname', 'Jason Day')]
[('sname', '1978 Tour de Suisse'), ('oname', 'Ueli Sutter')]
[('sname', "2011 UCI Road World Championships – Women's junior time trial"), ('oname', 'Elinor Barker')]
[('sname', "2012 UCI Road World Championships – Women's junior time trial"), ('oname', 'Elinor Barker')]
[('sname', '2014 Le Samyn'), ('oname', 'Maxime Vantomme')]
[('sname', '2014 The Oaks Club Challenger – singles'), ('oname', 'Anna Karolína Schmiedlová')]
10


In [43]:
# Q36593383 --> 'sports victory'
# P279 --> 'subclass of' ; Q18560095 -->'winner'
# P710 --> 'participant'
# Q8567 --> '2008 Summer Olympics' 
# P1687 --> 'Wikidata property' ; P1346 --> 'winner'
queryString = """
SELECT DISTINCT  ?sname ?oname 
WHERE {
{wd:Q15243387 ?p1 ?gold .
 ?gold ?p2 wd:Q36593383 .
 wd:Q36593383 ?p3 ?winner .
 ?winner ?p4 ?same .}
UNION
{
   # bind something
   ?same wdt:P1346 ?o .
   # get the label
   ?same sc:name ?sname .
   #?p sc:name ?pname .
   ?o sc:name ?oname .
}   
 }
#ORDER BY DESC (?oname)
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[]
[('sname', 'Miss Colombia 2007'), ('oname', 'Taliana Vargas')]
[('sname', "2013 European Diving Championships – men's 1 metre springboard"), ('oname', 'Illya Kvasha')]
[('sname', '2014 Tropicale Amissa Bongo'), ('oname', 'Egoitz García')]
[('sname', '2014 Tour of Qatar'), ('oname', 'Guillaume Van Keirsbulck')]
[('sname', '2014 WGC-Accenture Match Play Championship'), ('oname', 'Jason Day')]
[('sname', '1978 Tour de Suisse'), ('oname', 'Ueli Sutter')]
[('sname', "2011 UCI Road World Championships – Women's junior time trial"), ('oname', 'Elinor Barker')]
[('sname', "2012 UCI Road World Championships – Women's junior time trial"), ('oname', 'Elinor Barker')]
[('sname', '2014 Le Samyn'), ('oname', 'Maxime Vantomme')]
[('sname', '2014 The Oaks Club Challenger – singles'), ('oname', 'Anna Karolína Schmiedlová')]
[('sname', "1970 US Open – Men's Doubles"), ('oname', 'Pierre Barthès')]
[('sname', '2013 Czech Cycling Tour'), ('oname', 'Kristian Haugaard')]
[('sname', 'Tour du Rwanda

In [24]:
#NOT NOT
# Q15243387 --> 'Olympic gold medal'
# Q406039 --> 'gold medal'
# Q36593383 --> 'sports victory'
# Q18560095 --> 'winner'
# Q215627 --> 'person'
queryString = """
SELECT DISTINCT  ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q215627 ?p ?o .
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname
 }
#ORDER BY DESC (?oname)
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q830077'), ('oname', 'subject')]
[('pname', 'has quality'), ('o', 'http://www.wikidata.org/entity/Q61002'), ('oname', 'pseudonym')]
[('pname', 'opposite of'), ('o', 'http://www.wikidata.org/entity/Q8425'), ('oname', 'society')]
[('pname', 'has quality'), ('o', 'http://www.wikidata.org/entity/Q202444'), ('oname', 'given name')]
[('pname', 'has quality'), ('o', 'http://www.wikidata.org/entity/Q37787614'), ('oname', 'person property')]
[('pname', 'has quality'), ('o', 'http://www.wikidata.org/entity/Q641118'), ('oname', 'personality')]
[('pname', 'has quality'), ('o', 'http://www.wikidata.org/entity/Q2867027'), ('oname', 'personal life')]
[('pname', 'on focus list of Wikimedia project'), ('o', 'http://www.wikidata.org/entity/Q13382529'), ('oname', 'WikiProject Human rights')]
[('pname', 'has quality'), ('o', 'http://www.wikidata.org/entity/Q185836'), ('oname', 'age of a person')]
[('pname', 'has quality'), ('o', 'htt

In [23]:
# Q15243387 --> 'Olympic gold medal'
# Q406039 --> 'gold medal'
# Q36593383 --> 'sports victory'
# Q18560095 --> 'winner'
# Q215627 --> 'person'
# Q202444 --> 'given name'
queryString = """
SELECT DISTINCT  ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q202444 ?p ?o .
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname
 }
#ORDER BY DESC (?oname)
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P31'), ('oname', 'instance of')]
[('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1705'), ('oname', 'native label')]
[('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q10856962'), ('oname', 'anthroponym')]
[('pname', 'model item'), ('o', 'http://www.wikidata.org/entity/Q429948'), ('oname', 'Laura')]
[('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P407'), ('oname', 'language of work or name')]
[('pname', 'part of'), ('o', 'http://www.wikidata.org/entity/Q1071027'), ('oname', 'personal name')]
[('pname', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q1768721'), ('oname', 'Gujin Tushu Jicheng')]
[('pname', 'model item'), ('o', 'http://www.wikidata.org/entity/Q18002399'), ('oname', 'Martin')]
[('pname', 'disjoint union of'), ('o', 'http://www.wikidata.org/entity/Q23766486'), ('oname', 'list values as qualifiers')]
[

In [22]:
# Q15243387 --> 'Olympic gold medal'
# Q406039 --> 'gold medal'
# Q36593383 --> 'sports victory'
# Q18560095 --> 'winner'
# Q215627 --> 'person'
# Q202444 --> 'given name'
# Q1071027 --> 'personal name'
queryString = """
SELECT DISTINCT  ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q1071027 ?p ?o .
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname
 }
#ORDER BY DESC (?oname)
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('pname', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P1559'), ('oname', 'name in native language')]
[('pname', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1559'), ('oname', 'name in native language')]
[('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q10856962'), ('oname', 'anthroponym')]
[('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q38667285'), ('oname', 'full object name')]
[('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q82799'), ('oname', 'name')]
[('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q202444'), ('oname', 'given name')]
[('pname', 'disjoint union of'), ('o', 'http://www.wikidata.org/entity/Q23766486'), ('oname', 'list values as qualifiers')]
[('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q3702971'), ('oname', 'personal data')]
[('pname', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q6633985'), ('oname', 'Category:Human na

In [21]:
# Q15243387 --> 'Olympic gold medal'
# Q406039 --> 'gold medal'
# Q36593383 --> 'sports victory'
# Q18560095 --> 'winner'
# Q215627 --> 'person'
# Q202444 --> 'given name'
# Q1071027 --> 'personal name'
# Q82799 --> 'name'
queryString = """
SELECT DISTINCT  ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q82799 ?p ?o .
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname
 }
#ORDER BY DESC (?oname)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('pname', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P1559'), ('oname', 'name in native language')]
[('pname', 'said to be the same as'), ('o', 'http://www.wikidata.org/entity/Q151411'), ('oname', 'designation')]
[('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q8171'), ('oname', 'word')]
[('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q3695082'), ('oname', 'sign')]
[('pname', 'part of'), ('o', 'http://www.wikidata.org/entity/Q315'), ('oname', 'language')]
[('pname', 'manifestation of'), ('o', 'http://www.wikidata.org/entity/Q600590'), ('oname', 'n-tuple')]
[('pname', 'said to be the same as'), ('o', 'http://www.wikidata.org/entity/Q783521'), ('oname', 'title')]
[('pname', 'different from'), ('o', 'http://www.wikidata.org/entity/Q503992'), ('oname', 'nominal')]
[('pname', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P2561'), ('oname', 'name')]
[('pname', "topic's main category"), ('o', 'http://www.wikidata.org/en

In [2]:
# Q15243387 --> 'Olympic gold medal'
# Q406039 --> 'gold medal'
# Q36593383 --> 'sports victory'
# Q18560095 --> 'winner'
# Q215627 --> 'person'
queryString = """
SELECT DISTINCT  ?p ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q15243387 ?p ?o .
   # get the label
   ?p sc:name ?pname .
   ?o sc:name ?oname
 }
#ORDER BY DESC (?oname)
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q406039'), ('oname', 'gold medal')]
[('p', 'http://www.wikidata.org/prop/direct/P186'), ('pname', 'made from material'), ('o', 'http://www.wikidata.org/entity/Q1090'), ('oname', 'silver')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q636830'), ('oname', 'Olympic medal')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q636830'), ('oname', 'Olympic medal')]
[('p', 'http://www.wikidata.org/prop/direct/P186'), ('pname', 'made from material'), ('o', 'http://www.wikidata.org/entity/Q897'), ('oname', 'gold')]
[('p', 'http://www.wikidata.org/prop/direct/P3729'), ('pname', 'next lower rank'), ('o', 'http://www.wikidata.org/entity/Q15889641'), ('oname', 'Olympic silver medal')]
[('p', 'http://www.wikidata.org/prop/direct/P461'), ('pname

In [17]:
# Q8567 --> '2008 Summer Olympics' 
# P1687 --> 'Wikidata property' ; P1346 --> 'winner'
# P1344 --> 'participant in'

queryString = """
SELECT DISTINCT  ?b ?bname ?awardname
WHERE {
   # bind something
   ?b wdt:P1344 wd:Q8567;
   wdt:P1346 ?award.
   
   # get the label
   ?b sc:name ?bname .
   #?award sc:name ?awardname
 }
#ORDER BY DESC (?oname)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('b', 'http://www.wikidata.org/entity/Q288333'), ('pname', 'Tania Cagnotto'), ('award', 'http://www.wikidata.org/entity/Q106814418')]
[('b', 'http://www.wikidata.org/entity/Q109868'), ('pname', 'Birgit Prinz'), ('award', 'http://www.wikidata.org/entity/Q19323')]
2


 Final query for this task

In [ ]:
# write your final query

There is no any gold madel related data of 2008 Summer Olympic

In [18]:
# Q15243387 --> 'Olympic gold medal'
# Q406039 --> 'gold medal'
# P1344 --> 'participant in'
# Q36593383 --> 'sports victory'
# Q18560095 --> 'winner'
# Q215627 --> 'person'
# Q36593383 --> 'sports victory'
# P279 --> 'subclass of' ; Q18560095 -->'winner'
# P710 --> 'participant'
# Q8567 --> '2008 Summer Olympics' 
# P1687 --> 'Wikidata property' ; P1346 --> 'winner'
# P166 --> 'award received'
queryString = """
SELECT DISTINCT  ?b ?pname ?award
WHERE {
   # bind something
   ?b wdt:P1344 wd:Q8567;
   wdt:P1346 ?award.
   
   # get the label
   ?b sc:name ?pname .
   #?a sc:name ?oname
 }
#ORDER BY DESC (?oname)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('b', 'http://www.wikidata.org/entity/Q288333'), ('pname', 'Tania Cagnotto'), ('award', 'http://www.wikidata.org/entity/Q106814418')]
[('b', 'http://www.wikidata.org/entity/Q109868'), ('pname', 'Birgit Prinz'), ('award', 'http://www.wikidata.org/entity/Q19323')]
2


In [28]:
# write your queries
# Q5389 --> 'Olympic Games' 
# Q15243387 --> 'Olympic gold medal'
# Q406039 --> 'gold medal'
# P1344 --> 'participant in' 
# Q36593383 --> 'sports victory'
# Q18560095 --> 'winner'
# Q215627 --> 'person'
# Q36593383 --> 'sports victory'
# P279 --> 'subclass of' ; Q18560095 -->'winner'
# P710 --> 'participant'
# Q8567 --> '2008 Summer Olympics' 
# P1687 --> 'Wikidata property' ; P1346 --> 'winner'
# P166 --> 'award received' 

queryString = """
SELECT DISTINCT  ?participant_name
WHERE {
   # bind something
   ?s wdt:P31 wd:Q8567.
   ?e wdt:P1344 ?s.
   ?s wdt:P166 wd:Q15243387.
   
   ?e sc:name ?participant_name.
   filter(Regex(STR(?team_name) , "Summer*"))
   
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)
    


Results
Empty


## Task 7
Find athletes who won at least one gold medal in 4 different editions of the Summer Olympic Games (the result set must be a list of triples athlete IRI, label and #edition).

In [28]:
# write your queries
# Q5389 --> 'Olympic Games' 
# Q15243387 --> 'Olympic gold medal'
# Q406039 --> 'gold medal'
# P1344 --> 'participant in' 
# Q36593383 --> 'sports victory'
# Q18560095 --> 'winner'
# Q215627 --> 'person'
# Q36593383 --> 'sports victory'
# P279 --> 'subclass of' ; Q18560095 -->'winner'
# P710 --> 'participant'
# Q8567 --> '2008 Summer Olympics' 
# P1687 --> 'Wikidata property' ; P1346 --> 'winner'
# P166 --> 'award received'
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?oname ?participant_name
WHERE {
   # bind something
   wd:Q1189 ?p ?obj .
   # get the label
   ?p sc:name ?pname.
   ?obj sc:name ?oname.
   
   #?s wdt:P31 ?obj.
   #?e wdt:P1344 ?s.
   
   ?e sc:name ?participant_name.
   filter(Regex(?oname , "Summer*"))
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)
    


Results
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pname', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q2313564'), ('oname', "athletics at the 2004 Summer Olympics – men's 200 metres"), ('participant_name', 'software version identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pname', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q2313564'), ('oname', "athletics at the 2004 Summer Olympics – men's 200 metres"), ('participant_name', 'tabular software version')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pname', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q2313564'), ('oname', "athletics at the 2004 Summer Olympics – men's 200 metres"), ('participant_name', 'has edition or translation')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pname', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q2313564'), ('oname', "athletics at the 2004 Summer Olympics – men's 200 metres"), ('participant_nam

In [44]:
# write your queries
# Q5389 --> 'Olympic Games' 
# Q15243387 --> 'Olympic gold medal'
# Q406039 --> 'gold medal'
# P1344 --> 'participant in' 
# Q36593383 --> 'sports victory'
# Q18560095 --> 'winner'
# Q215627 --> 'person'
# Q36593383 --> 'sports victory'
# P279 --> 'subclass of' ; Q18560095 -->'winner'
# P710 --> 'participant'
# Q8567 --> '2008 Summer Olympics' 
# P1687 --> 'Wikidata property' ; P1346 --> 'winner'
# P166 --> 'award received'
queryString = """
SELECT DISTINCT ?pname ?obj ?oname ?participant_name
WHERE {
   # bind something
   ?tname wdt:P279 wd:Q5389 .
   # get the label
   ?tname sc:name ?team_name.
   
   ?s wdt:P31 ?tname.
   ?e wdt:P1344 ?s.
   
   ?e sc:name ?participant_name.
   filter(Regex(STR(?team_name) , "Summer*"))
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)
    


Results
[('team_name', 'Summer Olympic Games'), ('participant_name', 'Joseph Goubert')]
[('team_name', 'Summer Olympic Games'), ('participant_name', 'Archie Hahn')]
[('team_name', 'Summer Olympic Games'), ('participant_name', 'Josef Klíma')]
[('team_name', 'Summer Olympic Games'), ('participant_name', 'Jarl Johnsen')]
[('team_name', 'Summer Olympic Games'), ('participant_name', 'Nancy Archibald')]
[('team_name', 'Summer Olympic Games'), ('participant_name', 'Jan Vrolijk')]
[('team_name', 'Summer Olympic Games'), ('participant_name', 'Vincens Grandjean')]
[('team_name', 'Summer Olympic Games'), ('participant_name', 'Hans Spiegel')]
[('team_name', 'Summer Olympic Games'), ('participant_name', 'Teodoro Alcalde')]
[('team_name', 'Summer Olympic Games'), ('participant_name', 'Andrew Drew')]
[('team_name', 'Summer Olympic Games'), ('participant_name', 'Chris Forney')]
[('team_name', 'Summer Olympic Games'), ('participant_name', 'N. M. Smith')]
[('team_name', 'Summer Olympic Games'), ('partic

In [44]:
# write your queries
# Q5389 --> 'Olympic Games'
# P1344 --> 'participant in'
# P166 --> 'award received'
# Q15243387 --> 'Olympic gold medal'
queryString = """
SELECT DISTINCT ?pname ?obj ?oname ?participant_name
WHERE {
   # bind something
   ?tname wdt:P279 wd:Q5389 .  #get sub class of olympic games which is related to summer
   # get the label
   ?tname sc:name ?team_name.
   
   ?s wdt:P31 ?tname.
   ?e wdt:P1344 ?s.
   ?s wdt:P166 wd:Q15243387.
   
   ?e sc:name ?participant_name.
   filter(Regex(STR(?team_name) , "Summer*"))
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)
    


Results
[('team_name', 'Summer Olympic Games'), ('participant_name', 'Joseph Goubert')]
[('team_name', 'Summer Olympic Games'), ('participant_name', 'Archie Hahn')]
[('team_name', 'Summer Olympic Games'), ('participant_name', 'Josef Klíma')]
[('team_name', 'Summer Olympic Games'), ('participant_name', 'Jarl Johnsen')]
[('team_name', 'Summer Olympic Games'), ('participant_name', 'Nancy Archibald')]
[('team_name', 'Summer Olympic Games'), ('participant_name', 'Jan Vrolijk')]
[('team_name', 'Summer Olympic Games'), ('participant_name', 'Vincens Grandjean')]
[('team_name', 'Summer Olympic Games'), ('participant_name', 'Hans Spiegel')]
[('team_name', 'Summer Olympic Games'), ('participant_name', 'Teodoro Alcalde')]
[('team_name', 'Summer Olympic Games'), ('participant_name', 'Andrew Drew')]
[('team_name', 'Summer Olympic Games'), ('participant_name', 'Chris Forney')]
[('team_name', 'Summer Olympic Games'), ('participant_name', 'N. M. Smith')]
[('team_name', 'Summer Olympic Games'), ('partic

Final query for this task

In [ ]:
# write your final query

## Task 8
Consider all the edition of the Summer Olympic Games. For each sport return the athlete who won more gold medal (the result set must be a list of elements with sport IRI and label, athlete IRI and label, and #gold medal).

In [ ]:
# write your queries

Final query for this task

In [ ]:
# write your final query